## Custom API Notebook

### Initialize the Node

In [ ]:
# stdlib
from typing import Any

# syft absolute
import syft as sy
from syft import SyftSuccess

node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True, reset=True)
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_client.register(
    email="user@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="New User",
)
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

### Create a public custom API Endpoint by using the decorator

This allows node admin to create a new public endpoint by using only the decorator.

In [ ]:
@sy.api_endpoint(
    path="first.query",
    settings={"key": "value"},
)
def public_endpoint_method(
    context,
    query: str,
) -> Any:
    return context.settings["key"] == "value"


# Add it to the node.
response = domain_client.api.services.api.add(endpoint=public_endpoint_method)
assert isinstance(response, SyftSuccess)

In [ ]:
assert len(domain_client.api.services.api.api_endpoints()) == 1

In [ ]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

In [ ]:
assert domain_client.api.services.first.query(query="SELECT *")
assert domain_guest.api.services.first.query(query="SELECT *")

### Create public/private Custom API Endpoint using TwinAPIEndpoint

This allows the admin to create a public/private endpoint interface where the users can iteract with.

In [ ]:
@sy.public_api_endpoint(settings={"Hello": "Public"})
def public_function(
    context,
) -> str:
    return "Public Function Execution"


@sy.private_api_endpoint(settings={"Hello": "Private"})
def private_function(
    context,
) -> str:
    return "Private Function Execution"


new_endpoint = sy.TwinAPIEndpoint(
    path="third.query",
    private=private_function,
    public=public_function,
    description="Lore ipsulum ...",
)

# # Add it to the node.
response = domain_client.api.services.api.add(endpoint=new_endpoint)

In [ ]:
domain_client.api.services.api.api_endpoints()

In [ ]:
assert isinstance(response, SyftSuccess)
assert len(domain_client.api.services.api.api_endpoints()) == 2

In [ ]:
# Once api refresh is done, remove this cell
domain_client = node.login(email="info@openmined.org", password="changethis")
domain_guest = node.login(email="user@openmined.org", password="verysecurepassword")

In [ ]:
assert domain_client.api.services.third.query() == "Private Function Execution"

In [ ]:
assert domain_guest.api.services.third.query() == "Public Function Execution"

In [ ]:
@sy.syft_function_single_use(
    endpoint=domain_client.api.services.third.query,
)
def job_function(endpoint):
    return endpoint.mock()


# Create a new project
new_project = sy.Project(
    name="My Cool UN Project",
    description="Hi, I want to calculate the trade volume in million's with my cool code.",
    members=[domain_client],
)

result = new_project.create_code_request(job_function, domain_client)
assert isinstance(result, SyftSuccess)

In [ ]:
domain_client.requests[-1].approve()

In [ ]:
domain_client.code.job_function(endpoint=domain_client.api.services.third.query)

In [ ]:
domain_client.api.services.third.query

In [ ]:
job_function(endpoint=domain_client.api.services.third.query, syft_no_node=True)

In [ ]:
result = domain_client.api.services.api.delete(endpoint_path="third.query")
assert isinstance(result, SyftSuccess)

In [ ]:
assert len(domain_client.api.services.api.api_endpoints()) == 1